In [1]:
import os
import pandas as pd
from systematic_review import *

### Screening Experiments

In [2]:
fname = "../extraction/data/pond/pond_screening2.csv"
df = pd.read_csv(fname, index_col=0)

In [5]:
df.loc[df.definition_bool == True]

,doi,chunk,abstract_bool,definition_bool,table_bool,definition
71,lakes2,9,True,True,False,"```json\n{\n ""definition"": ""Lakes were divide..."
88,ponds3,0,True,True,False,"```json\n{\n ""definition"": ""The context defin..."
92,ponds3,4,True,True,True,"```json\n{\n ""definition"": ""Temporary ponds l..."
119,definitions3,1,True,True,False,"```json\n{\n ""definition_type"": ""ponds"",\n ""..."
189,definitions2,3,True,True,False,"```json\n{\n ""definition"": ""Ponds are small s..."
225,definitions2,39,True,True,False,"```json\n{\n ""definition"": ""Small lakes are d..."
231,definitions2,45,True,True,False,"```json\n{\n ""definition"": ""ponds were waterb..."
246,definitions2,60,True,True,False,"```json\n{\n ""definition"": ""Small waterbodies..."
251,ponds2,1,True,True,False,"```json\n{\n ""definitions"": [\n {\n ""..."
252,ponds2,2,True,True,False,"```json\n{\n ""definition"": ""Karst ponds are o..."


In [2]:
fname = "../extraction/data/coastal/screening4.csv"
df = pd.read_csv(fname, index_col=0)

In [21]:
# At least one of the relevant columns is True
relevant = df.loc[df['definition_bool'] | df['table_bool'] | df['measurement_bool']]
print(f"Number of relevant papers: {len(relevant.doi.value_counts())}")

# Papers with a definition 
definitions = df.loc[df['definition_bool'] == True]
print(f"Number of papers with a definition: {len(definitions.doi.value_counts())}")

# Papers with a table
tables = df.loc[df['table_bool'] == True]
print(f"Number of papers with a table: {len(tables.doi.value_counts())}")

# Papers with a measurement
measurements = df.loc[df['measurement_bool'] == True]
print(f"Number of papers with a measurement: {len(measurements.doi.value_counts())}")

# Papers with a measurement or a definition 
definitions_or_measurements = df.loc[df['definition_bool'] | df['measurement_bool']]
print(f"Number of papers with a definition or a measurement: {len(definitions_or_measurements.doi.value_counts())}")

Number of relevant papers: 392
Number of papers with a definition: 74
Number of papers with a table: 381
Number of papers with a measurement: 152
Number of papers with a definition or a measurement: 190


In [24]:
definitions_or_measurements

,doi,chunk,abstract_bool,definition_bool,table_bool,measurement_bool,definition
75,10.1002/lno.11678,44,True,True,True,False,"The context studies **lagoons**, and defines t..."
125,10.1002/lno.12322,1,True,True,False,True,"The context studies **tidal marshes, mangroves..."
127,10.1002/lno.12322,3,True,True,False,False,The coastal ecosystems being studied are **tid...
131,10.1002/lno.12322,7,True,False,False,True,NaN
132,10.1002/lno.12322,8,True,False,False,True,NaN
...,...,...,...,...,...,...,...
14288,10.1002/lno.11962,31,True,False,False,True,NaN
14289,10.1002/lno.11962,32,True,False,False,True,NaN
14292,10.1002/lno.11962,35,True,False,True,True,NaN
14294,10.1002/lno.11962,37,True,False,True,True,NaN


In [22]:
sample = definitions_or_measurements.sample(10)

In [35]:
# Sample of papers with definition and a measurement
definitions_and_measurements = df.loc[df['definition_bool'] & df['measurement_bool']]

for i in range(len(definitions_and_measurements)):
    sample = definitions_and_measurements.iloc[i,:]
    print(f"DOI: {sample.doi}")
    print(f"Definition: {sample.definition}")
    print()

DOI: 10.1002/lno.12322
Definition: The context studies **tidal marshes, mangroves, and seagrass meadows**. 

Specifically, these ecosystems are defined by their effectiveness as **long-term carbon sinks**, comprising **50% of the total marine soil (sediment) organic carbon budget** despite covering only **1% of the global ocean surface**. They also have **high rates of plant primary production** and **slow rates of decomposition in reducing soils**.


DOI: 10.1002/lno.12241
Definition: Mangroves, salt marshes, and seagrass beds are being studied, and are defined as "blue carbon ecosystems," specifically as "hotspots of marine organic carbon burial." They are also characterized by their carbon sequestration rates (net amount of organic carbon accumulated each year) and magnitudes of their carbon stocks (integration of long-term carbon storage in sediment over time).

DOI: 10.1002/lno.12416
Definition: The context specifically defines **mangrove belts** as an ecosystem populated by **C 3

In [36]:
definitions_and_measurements

,doi,chunk,abstract_bool,definition_bool,table_bool,measurement_bool,definition
125,10.1002/lno.12322,1,True,True,False,True,"The context studies **tidal marshes, mangroves..."
1152,10.1002/lno.12241,0,True,True,False,True,"Mangroves, salt marshes, and seagrass beds are..."
2310,10.1002/lno.12416,24,True,True,True,True,The context specifically defines **mangrove be...
5892,10.1002/lno.11607,11,True,True,True,True,The coastal ecosystems being studied are **man...
7993,10.1002/lno.12346,24,True,True,False,True,Small near-shore systems with a catchment area...
10060,10.1002/lno.70008,34,True,True,False,True,The coastal ecosystems being studied are **art...
10314,10.1002/lno.70105,0,True,True,False,True,Tidal marshes are being studied. They are defi...
12949,10.1002/lno.11598,1,True,True,False,True,The coastal ecosystems being studied are **tid...
13748,10.1002/lno.11997,3,True,True,False,True,**Narragansett Bay (USA)** is studied and is d...


In [9]:
df.loc[df.definition_bool == True].iloc[0].definition

'The context studies **lagoons**, and defines them with the following quantitative attributes:\n\n*   **Lagoon area (A):** 4.1 × 10<sup>6</sup> m<sup>2</sup>\n*   **Average water depth (h):** 0.56 m\n*   **Lagoon volume (V):** 2.3 × 10<sup>6</sup> m<sup>3</sup>\n*   **Water flow from streams (Q str):** 2.2 × 10<sup>3</sup> m<sup>3</sup> d<sup>-1</sup>\n*   **Water outflow (Q out):** 6.7 × 10<sup>3</sup> m<sup>3</sup> d<sup>-1</sup>\n*   **Concentration of suspended particles (C sus):** 0.04 kg m<sup>-3</sup>'

In [26]:
print(df.loc[df.definition_bool == True].iloc[4].definition)

Mangroves are being studied, specifically defined by their carbon sequestration attributes:

*   They sequester between 1.9 and 2.6 PgC in organic carbon globally in the top 1 m of soil.
*   The soil organic carbon (SOC) burial rate is estimated to be 20.2 TgC yr À1 for mangroves worldwide.
*   The estimated outwelling of dissolved inorganic carbon (DIC) from mangrove soil to the ocean is 366 TgC yr À1.
*   They are characterized by burrowing activity in the soil and tidal pumping.



</start_of_turn>
